In [3]:
import numpy as np
import cv2 as cv
import random
import sys
import dlib

# convert Dlib shape detector object to list of tuples
def dlibLandmarksToPoints(shape):
  points = []
  for p in shape.parts():
    pt = (p.x, p.y)
    points.append(pt)
  return points


# detect facial landmarks in image
def getLandmarks(faceDetector, landmarkDetector, im, FACE_DOWNSAMPLE_RATIO = 1):
  points = []
  imSmall = cv.resize(im,None,
                       fx=1.0/FACE_DOWNSAMPLE_RATIO,
                       fy=1.0/FACE_DOWNSAMPLE_RATIO,
                       interpolation = cv.INTER_LINEAR)

  faceRects = faceDetector(imSmall, 0)

  if len(faceRects) > 0:
    maxArea = 0
    maxRect = None
    # TODO: test on images with multiple faces
    for face in faceRects:
      if face.area() > maxArea:
        maxArea = face.area()
        maxRect = [face.left(),
                   face.top(),
                   face.right(),
                   face.bottom()
                  ]

    rect = dlib.rectangle(*maxRect)
    scaledRect = dlib.rectangle(int(rect.left()*FACE_DOWNSAMPLE_RATIO),
                             int(rect.top()*FACE_DOWNSAMPLE_RATIO),
                             int(rect.right()*FACE_DOWNSAMPLE_RATIO),
                             int(rect.bottom()*FACE_DOWNSAMPLE_RATIO))

    landmarks = landmarkDetector(im, scaledRect)
    points = dlibLandmarksToPoints(landmarks)
  return points

def readPoints(path):
    '''Read points from .tem file'''
    # Create an array of points.
    points = []
    # Read points
    with open(path) as file:
        no_lines = int(file.readline())
        for i, line in enumerate(file):
            if 0 <= i < no_lines:
                x, y = line.split()
                points.append((int(float(x)), int(float(y))))

    return points


def applyAffineTransform(src, srcTri, dstTri, size):
    '''Apply affine transform calculated using srcTri and dstTri to src and output an image of size.'''
    # Given a pair of triangles, find the affine transform.
    warpMat = cv.getAffineTransform(np.float32(srcTri), np.float32(dstTri))

    # Apply the Affine Transform just found to the src image
    dst = cv.warpAffine(src, warpMat, (size[0], size[1]), None,
                        flags=cv.INTER_LINEAR, borderMode=cv.BORDER_REFLECT_101)

    return dst


def morphTriangle(img1, img2, img, t1, t2, t, alpha):
    '''Warps and alpha blends triangular regions from img1 and img2 to img'''
    # Find bounding rectangle for each triangle
    r1 = cv.boundingRect(np.float32([t1]))
    r2 = cv.boundingRect(np.float32([t2]))
    r = cv.boundingRect(np.float32([t]))

    # Offset points by left top corner of the respective rectangles
    t1Rect = []
    t2Rect = []
    tRect = []

    for i in range(0, 3):
        tRect.append(((t[i][0] - r[0]), (t[i][1] - r[1])))
        t1Rect.append(((t1[i][0] - r1[0]), (t1[i][1] - r1[1])))
        t2Rect.append(((t2[i][0] - r2[0]), (t2[i][1] - r2[1])))

    # Get mask by filling triangle
    mask = np.zeros((r[3], r[2], 3), dtype=np.float32)
    cv.fillConvexPoly(mask, np.int32(tRect), (1.0, 1.0, 1.0), 16, 0)

    # Apply warpImage to small rectangular patches
    img1Rect = img1[r1[1]:r1[1] + r1[3], r1[0]:r1[0] + r1[2]]
    img2Rect = img2[r2[1]:r2[1] + r2[3], r2[0]:r2[0] + r2[2]]

    size = (r[2], r[3])
    warpImage1 = applyAffineTransform(img1Rect, t1Rect, tRect, size)
    warpImage2 = applyAffineTransform(img2Rect, t2Rect, tRect, size)

    # Alpha blend rectangular patches
    imgRect = (1.0 - alpha) * warpImage1 + alpha * warpImage2

    # Copy triangular region of the rectangular patch to the output image
    img[r[1]:r[1]+r[3], r[0]:r[0]+r[2]] = img[r[1]:r[1] +
                                              r[3], r[0]:r[0]+r[2]] * (1 - mask) + imgRect * mask


def rect_contains(rect, point):
    '''Check if a point is inside a rectangle'''
    if point[0] < rect[0]:
        return False
    elif point[1] < rect[1]:
        return False
    elif point[0] > rect[2]:
        return False
    elif point[1] > rect[3]:
        return False
    return True


def draw_point(img, p, color):
    '''Draw a point'''
    cv.circle(img, p, 2, color, cv.FILLED, cv.LINE_AA, 0)


def draw_voronoi(img, subdiv):
    '''Draw voronoi diagram'''
    (facets, centers) = subdiv.getVoronoiFacetList([])

    for i in range(0, len(facets)):
        ifacet_arr = []
        for f in facets[i]:
            ifacet_arr.append(f)

        ifacet = np.array(ifacet_arr, np.int)
        color = (random.randint(0, 255), random.randint(
            0, 255), random.randint(0, 255))

        cv.fillConvexPoly(img, ifacet, color, cv.LINE_AA, 0)
        ifacets = np.array([ifacet])
        cv.polylines(img, ifacets, True, (0, 0, 0), 1, cv.LINE_AA, 0)
        cv.circle(img, (centers[i][0], centers[i][1]),
                  3, (0, 0, 0), cv.FILLED, cv.LINE_AA, 0)


def draw_delaunay(img, subdiv, delaunay_color):
    '''Draw delaunay triangles'''
    triangleList = subdiv.getTriangleList()
    size = img.shape
    r = (0, 0, size[1], size[0])

    for t in triangleList:
        pt1 = (t[0], t[1])
        pt2 = (t[2], t[3])
        pt3 = (t[4], t[5])

        if rect_contains(r, pt1) and rect_contains(r, pt2) and rect_contains(r, pt3):
            cv.line(img, pt1, pt2, delaunay_color, 1, cv.LINE_AA, 0)
            cv.line(img, pt2, pt3, delaunay_color, 1, cv.LINE_AA, 0)
            cv.line(img, pt3, pt1, delaunay_color, 1, cv.LINE_AA, 0)


def calculateDelaunayTriangles(rect, subdiv, points, img, win_delaunay, delaunay_color, draw=False):
    '''Calculate delanauy triangle'''

    # Insert points into subdiv
    for p in points:
        subdiv.insert((p[0], p[1]))

    # List of triangles. Each triangle is a list of 3 points (6 numbers)
    triangleList = subdiv.getTriangleList()
    print('triangleList', len(triangleList))

    # Find the indices of triangles in the points array
    delaunayTri = []

    for t in triangleList:
        pt = []
        pt.append((t[0], t[1]))
        pt.append((t[2], t[3]))
        pt.append((t[4], t[5]))

        pt1 = (int(t[0]),int(t[1]))
        pt2 = (int(t[2]),int(t[3]))
        pt3 = (int(t[4]),int(t[5]))
        if rect_contains(rect, pt1) and rect_contains(rect, pt2) and rect_contains(rect, pt3):
            ind = []
            for j in range(0, 3):
                for k in range(0, len(points)):
                    if(abs(pt[j][0] - points[k][0]) < 1.0 and abs(pt[j][1] - points[k][1]) < 1.0):
                        ind.append(k)

            if len(ind) == 3:
                delaunayTri.append((ind[0], ind[1], ind[2]))

            # Draw lines
            if draw:
                cv.line(img, pt1, pt2, delaunay_color, 1, cv.LINE_AA, 0)
                cv.line(img, pt2, pt3, delaunay_color, 1, cv.LINE_AA, 0)
                cv.line(img, pt3, pt1, delaunay_color, 1, cv.LINE_AA, 0)
                imgS = cv.resize(img, (413, 531))

    return delaunayTri


def main():
    # Variables
    filename1 = './pure/models/wg.jpg'
    filename2 = './pure/models/tr2.jpg'
    # Landmark model location
    PREDICTOR_PATH = "./shape_predictor_68_face_landmarks.dat"
    faceDetector = dlib.get_frontal_face_detector()
    landmarkDetector = dlib.shape_predictor(PREDICTOR_PATH)

    alpha = 0.5

    # Define window names
    win_delaunay = 'Delaunay Triangulation'
    win_voronoi = 'Voronoi Diagram'

    # Define colors for drawing.
    d_col = (255, 255, 255)
    p_col = (0, 0, 255)

    # Read images
    img1 = cv.imread(filename1)
    img2 = cv.imread(filename2)

    points1 = getLandmarks(faceDetector, landmarkDetector, cv.cvtColor(img1, cv.COLOR_BGR2RGB))
    points2 = getLandmarks(faceDetector, landmarkDetector, cv.cvtColor(img2, cv.COLOR_BGR2RGB))
    points1 = np.array(points1)
    points2 = np.array(points2)


    img1_copy = cv.imread(filename1)
    img2_copy = cv.imread(filename2)

    # Read array of corresponding points
    #points1 = readPoints(filename1[:-4] + '.tem')
    #points2 = readPoints(filename2[:-4] + '.tem')
    points = []

    # Compute weighted average point coordinates
    for i in range(0, len(points1)):
        x = (1 - alpha) * points1[i][0] + alpha * points2[i][0]
        y = (1 - alpha) * points1[i][1] + alpha * points2[i][1]
        points.append((x, y))

    # Allocate space for final output
    imgMorph = np.zeros(img1.shape, dtype=img1.dtype)

    # Rectangle to be used with Subdiv2D
    size = img1.shape
    print(size)
    rect = (0, 0, size[1],  size[0])

    # Create an instance of Subdiv2D
    subdiv = cv.Subdiv2D(rect)

    # Calculate and draw delaunay triangles
    delaunayTri = calculateDelaunayTriangles(rect, subdiv, points, img1_copy, win_delaunay, d_col, draw=True)

    # Allocate space for Voronoi Diagram
    img_voronoi = np.zeros(img1.shape, dtype=img1.dtype)

    # Draw Voronoi diagram
    draw_voronoi(img_voronoi, subdiv)

    cv.namedWindow("Morph",cv.WINDOW_NORMAL)

    # Morph by reading calculated triangles
    for line in delaunayTri:
        x, y, z = line

        x = int(x)
        y = int(y)
        z = int(z)

        t1 = [points1[x], points1[y], points1[z]]
        t2 = [points2[x], points2[y], points2[z]]
        t  = [points[x],  points[y],  points[z]]

        # Morph one triangle at a time.
        morphTriangle(img1, img2, imgMorph, t1, t2, t, alpha)
        #cv.imshow("Morph",imgMorph)
        #cv.waitKey(0)
    cv.imshow("Morph",imgMorph)
    cv.waitKey(0)

    # Resize images
    desired_size = (860, 860)
    img1_copyS   = cv.resize(img1_copy,   desired_size)
    img2_copyS   = cv.resize(img2_copy,   desired_size)
    img1S        = cv.resize(img1,        desired_size)
    img2S        = cv.resize(img2,        desired_size)
    img_voronoiS = cv.resize(img_voronoi, desired_size)
    imgMorphS    = cv.resize(imgMorph,    desired_size)

    # Save Images
    cv.imwrite("Image_1.jpg", img1S)
    cv.imwrite("Image_2.jpg", img2S)
    cv.imwrite("Image_1_copy.jpg", img1_copyS)
    cv.imwrite("Image_2_copy.jpg", img2_copyS)
    cv.imwrite("Morphed_Face.jpg", imgMorphS)

if __name__ == "__main__":
    main()

(262, 193, 3)
triangleList 112


<ipython-input-3-3461e4bf9414>:140: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  ifacet = np.array(ifacet_arr, np.int)


error: OpenCV(4.5.4) :-1: error: (-5:Bad argument) in function 'circle'
> Overload resolution failed:
>  - Can't parse 'center'. Sequence item with index 0 has a wrong type
>  - Can't parse 'center'. Sequence item with index 0 has a wrong type
